In [2]:
import numpy as np
import random
import tqdm

import torch
import torch.nn.functional as F
from torch import nn
from torch import optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

from trainers import trainTorch, advTrain
from evalAttacks import evalClean, evalFGSMEnsem, evalAdvDiffModel, evalCombined
from cleverhans.future.torch.attacks.fast_gradient_method import fast_gradient_method

# #Different Attacks
# from evalAttacks import CWAttackRandomEnsem, CWAttackEnsem

In [3]:
nb_epochs = 30
batch_size = 128
learning_rate = 0.001
CIFARtrain = datasets.CIFAR10('data', train=True, download=True, transform=transforms.ToTensor())
CIFARtest = datasets.CIFAR10('data', train=False, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(CIFARtrain, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(CIFARtest,batch_size=batch_size)

Files already downloaded and verified


In [4]:
# Loading models from modelZoo
from modelZooCifar import CIFARNet,VWNet
CiNet1 = CIFARNet()
CiNet2 = CIFARNet()
CiNet3 = CIFARNet()

ckptPath = 'cifarCkpts/'

CiNet1.load_state_dict(torch.load(ckptPath + 'CIFARNet1.pth'))
CiNet2.load_state_dict(torch.load(ckptPath + 'CIFARNet2.pth'))
CiNet3.load_state_dict(torch.load(ckptPath + 'CIFARNet3.pth'))
models = [CiNet1, CiNet2, CiNet3]
# if torch.cuda.is_available():
#     CiNet1 = CiNet1.cuda()
#     CiNet2 = CiNet2.cuda()
#     CiNet3 = CiNet3.cuda()

In [5]:
class VotingSetFGSM(Dataset):
    def __init__(self, isTrain):
        if isTrain:
            self.imgDataset = CIFARtrain
        else:
            self.imgDataset = CIFARtest
        
    def __len__(self):
        return len(self.imgDataset)
        
    def __getitem__(self, idx):
        img = self.imgDataset[idx][0]
        selectedId = random.randint(0, len(models)-1)
        fgsm_model = models[selectedId]
        img = fast_gradient_method(fgsm_model, img.unsqueeze(0), eps=0.3, norm=np.inf, clip_min=0., clip_max=1.)
        target = torch.Tensor([1/(len(models)-1) for i in range(len(models))])
        target[selectedId] = 0.
        return img.squeeze(0), target
    

In [ ]:
# Voting Training
votingNet = VWNet(outputSize = 3)
votingNet = votingNet.cuda()

FGSMVoting = VotingSetFGSM(isTrain=True)
voting_trainloader = torch.utils.data.DataLoader(FGSMVoting, batch_size=batch_size, shuffle=True)
FGSMVotingTest = VotingSetFGSM(isTrain=False)
voting_testloader = torch.utils.data.DataLoader(FGSMVotingTest, batch_size=batch_size, shuffle=True)

optimizer = optim.Adam(votingNet.parameters(), lr=learning_rate)



bestLoss = -1
for epoch in range(nb_epochs):
    votingNet.train()
    for idx, (inpt, target) in tqdm.notebook.tqdm(enumerate(voting_trainloader)):
        inpt = inpt.cuda()
        target = target.cuda()
        pred = votingNet(inpt)
        cos_loss = torch.mean(1-torch.nn.CosineSimilarity()(pred, target))
        loss = cos_loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if idx % 100 == 0:
            print("At iteration {}:".format(idx))
            print("Prediction is {}".format(pred[0]))
            print("target is {}".format(target[0]))
            print("Current Loss is {}".format(loss))
    
    # Evaluation
    votingNet.eval()
    torch.save(votingNet.state_dict(), 'vNetCkpt/epoch'+ str(epoch)+'.pth')
    val_loss = 0
    for inpt, target in tqdm.notebook.tqdm(voting_testloader):
        with torch.no_grad():
            inpt = inpt.cuda()
            target = target.cuda()
            pred = votingNet(inpt)
            val_loss += torch.mean(1-torch.nn.CosineSimilarity()(pred, target))
    print("Validation Loss at Epoch {}: {}".format(epoch, val_loss/len(voting_testloader)))
    if bestLoss == -1 or val_loss < bestLoss:
        torch.save(votingNet.state_dict(), 'vNetCkpt/bestCkpt.pth')
        bestLoss = val_loss
        

At iteration 0:
Prediction is tensor([0.3191, 0.3435, 0.3374], device='cuda:0', grad_fn=<SelectBackward>)
target is tensor([0.5000, 0.5000, 0.0000], device='cuda:0')
Current Loss is 0.1846613883972168
At iteration 100:
Prediction is tensor([0.6292, 0.3564, 0.0144], device='cuda:0', grad_fn=<SelectBackward>)
target is tensor([0.5000, 0.5000, 0.0000], device='cuda:0')
Current Loss is 0.07946871221065521
At iteration 200:
Prediction is tensor([0.4538, 0.4983, 0.0479], device='cuda:0', grad_fn=<SelectBackward>)
target is tensor([0.5000, 0.5000, 0.0000], device='cuda:0')
Current Loss is 0.0320785716176033
At iteration 300:
Prediction is tensor([0.4694, 0.4943, 0.0364], device='cuda:0', grad_fn=<SelectBackward>)
target is tensor([0.5000, 0.5000, 0.0000], device='cuda:0')
Current Loss is 0.01551750022917986




Validation Loss at Epoch 0: 0.012064480222761631


At iteration 0:
Prediction is tensor([5.0371e-01, 4.7038e-04, 4.9582e-01], device='cuda:0',
       grad_fn=<SelectBackward>)
target is tensor([0.5000, 0.0000, 0.5000], device='cuda:0')
Current Loss is 0.008410144597291946
At iteration 100:
Prediction is tensor([5.2189e-01, 4.7779e-01, 3.2678e-04], device='cuda:0',
       grad_fn=<SelectBackward>)
target is tensor([0.5000, 0.5000, 0.0000], device='cuda:0')
Current Loss is 0.005602647550404072
At iteration 200:
Prediction is tensor([0.0909, 0.4556, 0.4535], device='cuda:0', grad_fn=<SelectBackward>)
target is tensor([0.0000, 0.5000, 0.5000], device='cuda:0')
Current Loss is 0.012246187776327133
At iteration 300:
Prediction is tensor([0.4812, 0.5176, 0.0011], device='cuda:0', grad_fn=<SelectBackward>)
target is tensor([0.5000, 0.5000, 0.0000], device='cuda:0')
Current Loss is 0.004674318712204695




Validation Loss at Epoch 1: 0.008003844879567623


At iteration 0:
Prediction is tensor([0.0472, 0.4998, 0.4530], device='cuda:0', grad_fn=<SelectBackward>)
target is tensor([0.0000, 0.5000, 0.5000], device='cuda:0')
Current Loss is 0.008038895204663277
At iteration 100:
Prediction is tensor([4.6405e-01, 7.2171e-06, 5.3594e-01], device='cuda:0',
       grad_fn=<SelectBackward>)
target is tensor([0.5000, 0.0000, 0.5000], device='cuda:0')
Current Loss is 0.005903128534555435
At iteration 200:
Prediction is tensor([0.4560, 0.5388, 0.0052], device='cuda:0', grad_fn=<SelectBackward>)
target is tensor([0.5000, 0.5000, 0.0000], device='cuda:0')
Current Loss is 0.0035653775557875633
At iteration 300:
Prediction is tensor([0.0288, 0.4807, 0.4905], device='cuda:0', grad_fn=<SelectBackward>)
target is tensor([0.0000, 0.5000, 0.5000], device='cuda:0')
Current Loss is 0.010614321567118168




Validation Loss at Epoch 2: 0.004324086010456085


At iteration 0:
Prediction is tensor([0.4903, 0.0258, 0.4839], device='cuda:0', grad_fn=<SelectBackward>)
target is tensor([0.5000, 0.0000, 0.5000], device='cuda:0')
Current Loss is 0.0021651778370141983
At iteration 100:
Prediction is tensor([4.8468e-01, 5.1525e-01, 6.5143e-05], device='cuda:0',
       grad_fn=<SelectBackward>)
target is tensor([0.5000, 0.5000, 0.0000], device='cuda:0')
Current Loss is 0.0007815398275852203
At iteration 200:
Prediction is tensor([0.0083, 0.4999, 0.4918], device='cuda:0', grad_fn=<SelectBackward>)
target is tensor([0.0000, 0.5000, 0.5000], device='cuda:0')
Current Loss is 0.0036894353106617928
At iteration 300:
Prediction is tensor([0.0086, 0.4965, 0.4949], device='cuda:0', grad_fn=<SelectBackward>)
target is tensor([0.0000, 0.5000, 0.5000], device='cuda:0')
Current Loss is 0.001468130387365818
